# 位置编码（Positional Encoding）学习内容

## 1. 为什么需要位置编码？

- Transformer 的 Self-Attention 机制本身对序列的顺序不敏感。
- 但自然语言是有顺序的，模型需要知道 token 的先后关系。
- 位置编码用于为每个 token 注入位置信息，使模型能利用顺序特征。

---

## 2. 常见的位置编码方法

### 2.1 绝对位置编码（Absolute Positional Encoding）

#### 2.1.1 正弦-余弦位置编码（Sinusoidal Positional Encoding）

- 论文《Attention is All You Need》中提出。
- 通过正弦和余弦函数为每个位置生成唯一的向量。
- 公式如下：

$$
PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right) \\
PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)
$$

- 其中 $pos$ 是位置，$i$ 是维度索引，$d_{model}$ 是嵌入维度。

#### 2.1.2 可学习的位置编码（Learnable Positional Embedding）

- 直接为每个位置分配一个可训练的向量，和词向量类似。
- 优点：灵活，能适应特定任务。
- 缺点：不能泛化到比训练时更长的序列。

---

### 2.2 相对位置编码（Relative Positional Encoding）

- 关注 token 之间的相对距离，而不是绝对位置。
- 适合处理长文本和泛化到不同长度的输入。
- 代表方法：Transformer-XL、T5、DeBERTa 等。

---



一、为什么需要位置编码？
1. 自注意力的缺陷
Transformer的自注意力机制通过计算所有词对之间的关联度（权重）捕捉全局依赖，但它本质上是位置无关的：

若将输入序列的词序打乱，自注意力计算结果不会改变（输出仅依赖词的内容，而非位置）。
示例：句子 "猫追狗" 和 "狗追猫" 的自注意力权重可能相同，但语义完全不同。

2. 位置编码的作用

核心目标：将位置信息显式注入模型，使模型能够区分序列中不同位置的词。
实现方式：为每个位置生成唯一的编码向量，与词向量相加（而非拼接）后输入模型。


二、Transformer的位置编码（正弦/余弦编码）
1. 数学公式
原始Transformer采用正弦和余弦函数的组合生成位置编码，公式如下：
PE(pos,2i)​=sin(100002i/dmodel​pos​)（偶数维度）
PE(pos,2i+1)​=cos(100002i/dmodel​pos​)（奇数维度）

参数说明：

pos：词在序列中的位置（从0开始）。
i：编码向量的维度索引（0 ≤ i < d_model/2）。
d_model：模型维度（与词向量维度相同）。



2. 直观理解

不同维度对应不同频率：高频（i较小时）捕捉局部位置变化，低频（i较大时）捕捉全局位置关系。
波长范围：波长从2π到20000π不等，覆盖不同距离的位置差异。
相对位置可线性表达：通过三角恒等式，模型能学习到相对位置关系（如位置pos+k的编码可用pos的编码线性表示）。



三、关键特性解析
1. 与词向量相加而非拼接

维度一致性：位置编码维度（d_model）与词向量维度一致，可直接相加。
信息融合：相加操作允许模型动态调整位置和语义的权重（如动词可能更依赖位置信息）。

2. 正弦/余弦的周期性优势

绝对位置编码：每个位置有唯一编码。
相对位置建模：对任意偏移量k，存在线性变换矩阵M，使得：
PEpos+k​=Mk​⋅PEpos​
这一性质使模型能通过注意力机制捕捉相对位置关系。


四、位置编码的变体方法
1. 可学习的位置嵌入（Learnable Positional Embedding）

实现方式：将位置编码视为可训练参数（类似词嵌入），随机初始化后通过反向传播学习。
优点：灵活适应任务需求。
缺点：需要更多数据，难以泛化到训练时未见的序列长度。
应用：BERT、GPT系列早期版本。

2. 相对位置编码（Relative Position Encoding）

核心思想：直接建模词对之间的相对位置关系。
实现方式：

在注意力计算中引入相对位置偏置项：
Attention(Q,K,V)=Softmax(dk​​QKT​+B)V
其中B是基于相对位置的偏置矩阵。
示例：Transformer-XL中的相对位置编码。


优点：更适合处理长文本，捕捉局部依赖。

3. 旋转位置编码（RoPE, Rotary Position Embedding）

核心思想：通过旋转矩阵将绝对位置编码融入注意力计算。
数学形式：

对查询向量Q和键向量K施加旋转：
Qm​=Q⋅Rm​,Kn​=K⋅Rn​
其中R_m, R_n是位置相关的旋转矩阵。


优点：保持相对位置的衰减性，适合长序列建模。
应用：LLaMA、ChatGLM等大模型。


五、关键问题解答
Q1: 为什么位置编码要与词向量相加？

答：相加操作使位置信息与语义信息在同一空间融合，模型能自适应调整两者的权重。若拼接会增加维度，导致计算量增大。

Q2: 正弦位置编码是否适用于所有任务？

答：不一定。对于短文本任务（如分类）效果显著，但对长文本（如书籍生成），相对位置编码（如ALiBi）可能更优。

Q3: 如何处理超出训练时最大长度的位置？

可学习的位置嵌入：无法处理，需要截断或外推。
正弦编码：可通过公式直接计算任意位置，但模型可能对远距离位置关系建模困难。
改进方法：位置插值（如Meta的LLaMA-2扩展上下文窗口）。


六、实际应用建议

预训练模型选择：

文本生成（如GPT）：优先使用旋转位置编码（RoPE）。
长文本理解（如法律文档）：选择ALiBi或相对位置编码。


自定义任务：

数据量小 → 使用正弦编码（避免过拟合）。
数据量大 → 可尝试可学习的位置嵌入。




附：ALiBi位置编码示例
ALiBi（Attention with Linear Biases）通过线性偏置项直接惩罚远距离词对的注意力权重：
Attention(Q,K,V)=Softmax(dk​​QKT​−m⋅∣i−j∣)V
其中m是斜率参数，|i-j|是相对距离。这种方法无需显式存储位置编码，更适合处理超长序列。



https://blog.csdn.net/v_JULY_v/article/details/134085503

In [ ]:
import torch
import math

def get_sinusoid_encoding_table(seq_len, d_model):
    table = torch.zeros(seq_len, d_model)
    for pos in range(seq_len):
        for i in range(0, d_model, 2):
            table[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
            if i + 1 < d_model:
                table[pos, i+1] = math.cos(pos / (10000 ** ((2 * i)/d_model)))
    return table

# 示例：生成长度为10，维度为16的位置编码
encoding = get_sinusoid_encoding_table(10, 16)
print(encoding)